# Matchbox Educable Noughts and Crosses Engine



Each box corresponds to a gamestate, and has a list of moves it can make and corresponding bead counts.

In [3]:
from dataclasses import dataclass


@dataclass
class Box:
    state: str      # state should be generic but dataclass does not seem to support generic types????????????????
    moves: list[str]
    beads: list[int]
    



The gamestate in naughts and crosses can be represented as a string of nine characters.
for example the string `XX-OX--OO"` represents the gamestate:

|X|X||
|---|---|---|
|O|X||
||O|O|

The entire MENACE machine can be thought of as a mapping from states to boxes:

In [ ]:
type MENACE = dict[str, Box[str]]

: 

In order to reduce the statespace we should ignore treat states that are symmetrical or rotationally symmetrical as redundant.
To do this we create functions for flipping and rotating the board:

In [ ]:
def flipX(board:str):
    "flips the board over the X axis"
    return board[6:9] + board[3:6] + board[:3]

def rotate(board:str):
    b = ""
    for i in range(0,3):
        for j in range(2,-1,-1):
            b += board[3*j + i]
    return b

def all_rotations(board: str):
    rs = [board]
    for _ in range(3):
        board = rotate(board)
        rs.append(board)
    return rs

def all_versions(board:str):
    return all_rotations(board) + all_rotations(flipX(board))

It will also be useful to have a function for checking whether a board is winning for a player:

In [ ]:
def winstate(state: str, player: str):
    wins = {"012","036","058","157","268","256","345","678"}
    for win in wins:
        for i in win:
            if state[int(i)] != player:
                break
        else: return True

In [ ]:
from collections import deque


def initialise_game(beads = 10) -> MENACE:
    d: MENACE = {}
    q = deque()
    q.append("---------")
    while q:
        s: str = q.popleft()
        if not sum(v in d for v in all_versions(s)):
            moves = [s[:i] + "O" + s[i+1:] for i in range(9) if s[i] == "-"]
            d[s] = Box(s, moves, [beads] * len(moves))
            for m in moves:
                if winstate(m, "O"): continue
                if not sum(v in d for v in all_versions(m)):
                    q.append(m)
    return d
    
print(len(initialise_game(10)))

In [ ]:
from random import choice, choices

def playturn(menace: MENACE, state: str) -> str:
    if state not in menace:
        raise(ValueError("Please check game state is valid"))
    
    box = menace[state]
    return choices(population = box.moves, weights = box.beads)[0]
    
    
@dataclass
class GameResult:
    win: bool
    states: list[str]

def randomturn(s:str) -> str:
    return choice([s[:i] + "X" + s[i+1:] for i in range(9) if s[i] == "-"])
    
    
def randomgame(menace: MENACE) -> GameResult:
    state = "---------"
    states = [state]
    while 1:
        state = playturn(menace, state)
        states.append(state)
        if winstate(state, "O"): return GameResult(True, states)
        state = randomturn(state)
        states.append(state)
        if winstate(state, "X"): return GameResult(False, states)
        
    raise(Exception("This is just to appease Pylance!!!"))